In [ ]:
# ResNet34 3ch

import os
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from glob import glob
from tqdm import tqdm
from torchvision.models import resnet34, ResNet34_Weights

# 환경 설정
SLICE_ROOT = "/data1/lidc-idri/slices"
BATCH_SIZE = 16
NUM_EPOCHS = 10
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 레이블 추출
def extract_label_from_filename(filename):
    try:
        score = int(filename.split("_")[-1].replace(".npy", ""))
        if score == 3: return None
        return 1 if score >= 4 else 0
    except:
        return None

# 파일 리스트 정리
all_files = glob(os.path.join(SLICE_ROOT, "LIDC-IDRI-*", "*.npy"))
file_label_pairs = [(f, extract_label_from_filename(f)) for f in all_files]
file_label_pairs = [(f, l) for f, l in file_label_pairs if l is not None]
files, labels = zip(*file_label_pairs)
train_files, val_files, train_labels, val_labels = train_test_split(files, labels, test_size=0.2, random_state=42)

# 3채널 Dataset 클래스
class LIDC3ChannelDataset(Dataset):
    def __init__(self, file_paths, labels, transform=None):
        self.file_paths = file_paths
        self.labels = labels
        self.transform = transform

    def __getitem__(self, idx):
        center_path = self.file_paths[idx]
        label = self.labels[idx]
        folder = os.path.dirname(center_path)
        fname = os.path.basename(center_path)
        slice_num = int(fname.split("_")[1])
        suffix = fname.split("_")[-1]

        # 주변 3개 슬라이스 로딩
        slice_indices = [slice_num - 1, slice_num, slice_num + 1]
        images = []
        for sn in slice_indices:
            path = os.path.join(folder, f"slice_{sn:03d}_{suffix}")
            if os.path.exists(path):
                img = np.load(path).astype(np.float32)
            else:
                img = np.load(center_path).astype(np.float32)
            img = (img - img.min()) / (img.max() - img.min() + 1e-8)
            images.append(img)

        stacked = np.stack(images, axis=0)
        img_tensor = torch.tensor(stacked)
        img_tensor = F.interpolate(img_tensor.unsqueeze(0), size=(224, 224), mode='bilinear', align_corners=False).squeeze(0)
        if self.transform:
            img_tensor = self.transform(img_tensor)
        return img_tensor, torch.tensor(label).float(), center_path

    def __len__(self):
        return len(self.file_paths)

# Transform
val_transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])

# Dataloader 구성
train_dataset = LIDC3ChannelDataset(train_files, train_labels, transform=val_transform)
val_dataset = LIDC3ChannelDataset(val_files, val_labels, transform=val_transform)
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE)

# 모델 정의 (ResNet34)
model = resnet34(weights=ResNet34_Weights.IMAGENET1K_V1)
model.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3, bias=False)  # 3채널용으로 수정
model.fc = nn.Linear(model.fc.in_features, 1)
model = model.to(DEVICE)

# 손실함수와 Optimizer
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)

# 학습 루프
best_acc = 0
for epoch in range(NUM_EPOCHS):
    model.train()
    running_loss = 0.0
    for imgs, labels, _ in tqdm(train_loader, desc=f"Epoch {epoch+1}/{NUM_EPOCHS}"):
        imgs, labels = imgs.to(DEVICE), labels.unsqueeze(1).to(DEVICE)
        optimizer.zero_grad()
        output = model(imgs)
        loss = criterion(output, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"[Epoch {epoch+1}] Loss: {running_loss / len(train_loader):.4f}")

    # 검증
    model.eval()
    correct, total = 0, 0
    all_preds, all_labels = [], []
    with torch.no_grad():
        for imgs, labels, _ in val_loader:
            imgs = imgs.to(DEVICE)
            labels = labels.to(DEVICE)
            out = model(imgs).squeeze()
            probs = torch.sigmoid(out)
            preds = (probs > 0.5).long()
            correct += (preds == labels).sum().item()
            total += labels.size(0)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    val_acc = correct / total
    print(f"Validation Accuracy: {val_acc:.4f}")
    if val_acc > best_acc:
        best_acc = val_acc
        torch.save(model.state_dict(), "best_model_resnet34_3ch.pth")
        print("✅ Best model saved!")

# 최종 리포트
print("\n📊 Classification Report:")
print(classification_report(all_labels, all_preds, digits=4))